In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def normalize_img(img):
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = 255 * (img + img_min) / (img_min + img_max)
    return img_norm.astype(int)


def normalize_to_one(img):
    img = np.array(img)
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = (img + img_min) / (img_min + img_max)
    return img_norm


def generate_single_octave_plane(width, grid_size, random_values=None, func=None):
    num_cells_x = width // grid_size
    
    if random_values is None:
        random_values = np.random.rand(num_cells_x + 1)

    noise_map = np.zeros(width)

    for x in range(width):
        cell_x = x // grid_size

        local_x = (x % grid_size) / grid_size

        left = random_values[cell_x]
        right = random_values[cell_x + 1]
        
        if func is None:
            smooth_x = local_x
        else:
            smooth_x = func(local_x)

        interp = left * (1 - smooth_x) + right * smooth_x

        noise_map[x] += interp

    return random_values, noise_map


def mesh_plane(x_min, x_max, x_step):
    x_lines = [0]

    for i in range(x_min, x_max, x_step):
        x_lines.append(x_lines[-1] + x_step)

    return x_lines


def qubic_interp(x):
    return x * x * (3 - 2 * x)


def fifth_order_interp(x):
    return 6 * x ** 5 - 15 * x ** 4 + 10 * x ** 3

In [ ]:
pic_size = 128
mesh_size = 16

random_values = np.random.rand(pic_size // mesh_size + 1)

In [ ]:
rand_vals, lin_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values)
rand_vals, qubic_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values, func=qubic_interp)

rand_vals_x = mesh_plane(0, pic_size - 1, mesh_size)
noise_x = mesh_plane(0, pic_size - 1, 1)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(noise_x, lin_noise)
ax.plot(noise_x, qubic_noise)
ax.scatter(rand_vals_x, rand_vals, marker='.')

ax.set_xticks(rand_vals_x)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.savefig('types_of_interp.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []
random_values_oct = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rv = np.random.rand(pic_size // freq + 1)
    random_values_oct.append(rv)
    
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=rv)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(f'lin_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=qubic_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(f'cubic_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=fifth_order_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(f'fifth_order_octave_sum.png')